# FP-Growth

FP-Growth（Frequent Pattern Growth， 频繁模式增长），它比Apriori算法效率更高，在整个算法执行过程中，只需要遍历数据集2次，就可完成频繁模式的发现

FP-growth算法基于Apriori构建，但采用了高级的数据结构减少扫描次数，大大加快了算法速度。FP-growth算法只需要对数据库进行两次扫描，而Apriori算法对于每个潜在的频繁项集都会扫描数据集判定给定模式是否频繁，因此FP-growth算法的速度要比Apriori算法快。

FP-growth算法发现频繁项集的基本过程如下：
- 构建FP树
- 从FP树中挖掘频繁项集

> FP-growth算法
  * 优点：一般要快于Apriori。
  * 缺点：实现比较困难，在某些数据集上性能会下降。
  * 适用数据类型：离散型数据。

 

### 1.  创建FP树的数据结构

In [1]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode=None):
        self.name=nameValue
        self.count=numOccur
        self.nodeLink=None
        self.parent = parentNode
        self.children = {}
        
    def inc(self, numOccur):
        self.count += numOccur
        
    def disp(self, ind=1):
        print ' '*ind, self.name, ' ', self.count
        for child in self.children.values():
            child.disp(ind+1)

每个树节点由五个数据项组成：
- name：节点元素名称，在构造时初始化为给定值
- count：出现次数，在构造时初始化为给定值
- nodeLink：指向下一个相似节点的指针，默认为None
- parent：指向父节点的指针，在构造时初始化为给定值
- children：指向子节点的字典，以子节点的元素名称为键，指向子节点的指针为值，初始化为空字典

成员函数：
- inc()：增加节点的出现次数值
- disp()：输出节点和子节点的FP树结构

In [2]:
rootNode = treeNode('pyramid', 9)

In [3]:
rootNode.children['eye'] = treeNode('eye', 13)

In [4]:
rootNode.disp()

  pyramid   9
   eye   13


In [5]:
rootNode.children['phoenix']=treeNode('phoenix', 3)
rootNode.disp()

  pyramid   9
   eye   13
   phoenix   3


算法：构建FP树

> 输入：数据集、最小值尺度
> 输出：FP树、头指针表
> 1. 遍历数据集，统计各元素项出现次数，创建头指针表
> 2. 移除头指针表中不满足最小值尺度的元素项
> 3. 第二次遍历数据集，创建FP树。对每个数据集中的项集：
>     - 3.1 初始化空FP树
>     - 3.2 对每个项集进行过滤和重排序
>     - 3.3 使用这个项集更新FP树，从FP树的根节点开始：
>         - 3.3.1 如果当前项集的第一个元素项存在于FP树当前节点的子节点中，则更新这个子节点的计数值
>         - 3.3.2 否则，创建新的子节点，更新头指针表
>         - 3.3.3 对当前项集的其余元素项和当前元素项的对应子节点递归3.3的过程

In [6]:
def createTree(dataSet, minSup=1):
    ''' 创建FP树 '''
    # 第一次遍历数据集，创建头指针表
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    # 移除不满足最小支持度的元素项
    for k in headerTable.keys():
        if headerTable[k] < minSup:
            del(headerTable[k])
    # 空元素集，返回空
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0:
        return None, None
    # 增加一个数据项，用于存放指向相似元素项指针
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
    retTree = treeNode('Null Set', 1, None) # 根节点
    # 第二次遍历数据集，创建FP树
    for tranSet, count in dataSet.items():
        localD = {} # 对一个项集tranSet，记录其中每个元素项的全局频率，用于排序
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0] # 注意这个[0]，因为之前加过一个数据项
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)] # 排序
            updateTree(orderedItems, retTree, headerTable, count) # 更新FP树
    return retTree, headerTable

需要注意的是，参数中的dataSet的格式比较奇特，不是直觉上得集合的list，而是一个集合的字典，以这个集合为键，值部分记录的是这个集合出现的次数。于是要生成这个dataSet还需要后面的createInitSet()函数辅助。因此代码中第7行中的dataSet[trans]实际获得了这个trans集合的出现次数（在本例中均为1），同样第21行的“for tranSet, count in dataSet.items():”获得了tranSet和count分别表示一个项集和该项集的出现次数。——这样做是为了适应后面在挖掘频繁项集时生成的条件FP树。


In [7]:
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        # 有该元素项时计数值+1
        inTree.children[items[0]].inc(count)
    else:
        # 没有这个元素项时创建一个新节点
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 更新头指针表或前一个相似元素项节点的指针指向新节点
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
 
    if len(items) > 1:
        # 对剩下的元素项迭代调用updateTree函数
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

为了让FP(Frequent Pattern)树 **生长**，需要调用 **updateTree**，其中输入参数为一个list。该函数首先测试事务中的第一个元素项是否作为子节点存在，如果存在，则更新该元素项的计数；如果不存在，则创建一个新的 **treeNode**并将其作为一个子节点添加到树中。这时，头指针表也要更新以指向新的节点。更新头指针表需要调用函数 **updateHeader**。**updateTree** 完成最后一件事是不断迭代调用自身，每次调用会去掉列表中的第一个元素。

In [8]:
def updateHeader(nodeToTest, targetNode):
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

这个函数其实只做了一件事，就是获取头指针表中该元素项对应的单链表的尾节点，然后将其指向新节点targetNode。

**updateHeader** 函数确保节点链接指向树中该元素项的每一个实例。从头指针表的nodeLink开始，一直源着nodeLink直到达到链表的末尾。这就是一个链表。当处理树的时候，和种很自然的反应就是迭代完成每一件事。当以相同方式处理链表时可能会遇到一些问题，原因是如果链表很长可能会遇到迭代用的次数限制。

### 生成数据集

In [9]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat
 
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [10]:
simpleDat = loadSimpDat()
simpleDat


[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [11]:
initSet = createInitSet(simpleDat)
initSet

{frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'h', 'j', 'p', 'r', 'z'}): 1}

In [12]:
myFPTree, myHeaderTab = createTree(initSet)
myFPTree.disp()

  Null Set   1
   x   1
    s   1
     r   1
      o   1
       n   1
   z   5
    x   3
     s   2
      t   2
       y   2
        q   1
         e   1
          m   1
        u   1
         w   1
          v   1
     r   1
      t   1
       y   1
        q   1
         p   1
    r   1
     p   1
      h   1
       j   1


上面给出的是元素及其对应的频率计数值，其中每个缩进表示所处树的深度。

现在我们已经构建了FP树，接下来就使用它是行频繁项集挖掘

**从FP树中抽取频繁项集的三个基本步骤如下:**

    1. 利用条件模式基
    2. 构建一个条件FP树；
    3. 迭代重复步骤1步骤2，直到树包含一个元素项为止。

In [13]:
def findPrefixPath(basePat, treeNode):
    ''' 创建前缀路径 '''
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

该函数代码用于为给定元素项生成一个条件模式基（前缀路径），这通过访问树中所有包含给定元素项的节点来完成。参数basePet表示输入的频繁项，treeNode为当前FP树种对应的第一个节点（可在函数外部通过headerTable[basePat][1]获取）。函数返回值即为条件模式基condPats，用一个字典表示，键为前缀路径，值为计数值。

In [14]:
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

这个函数直接修改prefixPath的值，将当前节点leafNode添加到prefixPath的末尾，然后递归添加其父节点。最终结果，prefixPath就是一条从treeNode（包括treeNode）到根节点（不包括根节点）的路径。在主函数findPrefixPath()中再取prefixPath[1:]，即为treeNode的前缀路径。

In [15]:
findPrefixPath('x', myHeaderTab['x'][1])
findPrefixPath('z', myHeaderTab['z'][1])
findPrefixPath('r', myHeaderTab['r'][1])

{frozenset({'x', 'z'}): 1, frozenset({'s', 'x'}): 1, frozenset({'z'}): 1}

## 递归查找频繁项集
有了FP树和条件FP树，我们就可以在前两步的基础上递归得查找频繁项集。

递归的过程是这样的：

输入：我们有当前数据集的FP树（inTree，headerTable）
1. 初始化一个空列表preFix表示前缀
2. 初始化一个空列表freqItemList接收生成的频繁项集（作为输出）
3. 对headerTable中的每个元素basePat（按计数值由小到大），递归：

        3.1 记basePat + preFix为当前频繁项集newFreqSet
        3.2 将newFreqSet添加到freqItemList中
        3.3 计算t的条件FP树（myCondTree、myHead）
        3.4 当条件FP树不为空时，继续下一步；否则退出递归
        3.4 以myCondTree、myHead为新的输入，以newFreqSet为新的preFix，外加freqItemList，递归这个过程

In [16]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1])]
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        myCondTree, myHead = createTree(condPattBases, minSup)
 
        if myHead != None:
            # 用于测试
            print 'conditional tree for:', newFreqSet
            myCondTree.disp()
 
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

**输入参数：**
- inTree和headerTable是由createTree()函数生成的数据集的FP树
- minSup表示最小支持度
- preFix请传入一个空集合（set([])），将在函数中用于保存当前前缀
- freqItemList请传入一个空列表（[]），将用来储存生成的频繁项集

In [19]:
freqItems = []
mineTree(myFPTree, myHeaderTab, 3, set([]), freqItems)
freqItems

conditional tree for: set(['s'])
  Null Set   1
   x   3
conditional tree for: set(['t'])
  Null Set   1
   x   3
    z   3
conditional tree for: set(['z', 't'])
  Null Set   1
   x   3
conditional tree for: set(['y'])
  Null Set   1
   x   3
    z   3
     t   3
conditional tree for: set(['y', 'z'])
  Null Set   1
   x   3
conditional tree for: set(['y', 't'])
  Null Set   1
   x   3
    z   3
conditional tree for: set(['y', 'z', 't'])
  Null Set   1
   x   3
conditional tree for: set(['x'])
  Null Set   1
   z   3


[{'e'},
 {'m'},
 {'o'},
 {'n'},
 {'u'},
 {'w'},
 {'v'},
 {'h'},
 {'j'},
 {'q'},
 {'p'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'x'},
 {'t', 'z'},
 {'t', 'x', 'z'},
 {'y'},
 {'x', 'y'},
 {'y', 'z'},
 {'x', 'y', 'z'},
 {'t', 'y'},
 {'t', 'x', 'y'},
 {'t', 'y', 'z'},
 {'t', 'x', 'y', 'z'},
 {'r'},
 {'x'},
 {'x', 'z'},
 {'z'}]

## 封装
至此，完整的FP-growth算法已经可以运行。封装整个过程如下：

In [22]:
def fpGrowth(dataSet, minSup=3):
    initSet = createInitSet(dataSet)
    myFPtree, myHeaderTab = createTree(initSet, minSup)
    freqItems = []
    mineTree(myFPtree, myHeaderTab, minSup, set([]), freqItems)
    return freqItems

**注意**，这里直接使用了上节中的createInitSet()函数，这里有个问题：上节中的loadSimpDat()函数返回了一组简单的样例数据，没有相同的事务，所以createInitSet()函数中直接赋值“retDict[frozenset(trans)] = 1”没有问题。但是如果要封装成一个通用的FP-growth算法，就还需要处理输入数据有相同事务的情形，createInitSet()函数中需要**累加** retDict[frozenset(trans)]。

In [23]:
dataSet = loadSimpDat()
freqItems = fpGrowth(dataSet)
freqItems

conditional tree for: set(['s'])
  Null Set   1
   x   3
conditional tree for: set(['y'])
  Null Set   1
   x   3
    z   3
conditional tree for: set(['y', 'z'])
  Null Set   1
   x   3
conditional tree for: set(['t'])
  Null Set   1
   y   3
    x   3
     z   3
conditional tree for: set(['x', 't'])
  Null Set   1
   y   3
conditional tree for: set(['z', 't'])
  Null Set   1
   y   3
    x   3
conditional tree for: set(['x', 'z', 't'])
  Null Set   1
   y   3
conditional tree for: set(['x'])
  Null Set   1
   z   3


[{'s'},
 {'s', 'x'},
 {'y'},
 {'x', 'y'},
 {'y', 'z'},
 {'x', 'y', 'z'},
 {'t'},
 {'t', 'y'},
 {'t', 'x'},
 {'t', 'x', 'y'},
 {'t', 'z'},
 {'t', 'y', 'z'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y', 'z'},
 {'r'},
 {'x'},
 {'x', 'z'},
 {'z'}]

## 总结
FP-growth算法是一种用于发现数据集中频繁模式的有效方法。FP-growth算法利用Apriori原则，执行更快。Apriori算法产生候选项集，然后扫描数据集来检查它们是否频繁。由于只对数据集扫描两次，因此FP-growth算法执行更快。在FP-growth算法中，数据集存储在一个称为FP树的结构中。FP树构建完成后，可以通过查找元素项的条件基及构建条件FP树来发现频繁项集。该过程不断以更多元素作为条件重复进行，直到FP树只包含一个元素为止。

FP-growth算法还有一个map-reduce版本的实现，它也很不错，可以扩展到多台机器上运行。Google使用该算法通过遍历大量文本来发现频繁共现词，其做法和我们刚才介绍的例子非常类似

## 示例：从新闻网站点击流中挖掘新闻报道
从新闻网站点击流中挖掘热门新闻报道。这是一个很大的数据集，有将近100万条记录（参见扩展阅读：kosarak）。在源数据集合保存在文件kosarak.dat中。该文件中的每一行包含某个用户浏览过的新闻报道。新闻报道被编码成整数，我们可以使用Apriori或FP-growth算法挖掘其中的频繁项集，查看那些新闻ID被用户大量观看到


In [25]:
parsedDat = [line.split() for line in open('kosarak.dat').readlines()]
initSet = createInitSet(parsedDat)
## 然后构建FP树，并从中寻找那些至少被10万人浏览过的新闻报道。
frequentList = fpGrowth(initSet,100000)

conditional tree for: set(['1'])
  Null Set   1
   6   107404
conditional tree for: set(['3'])
  Null Set   1
   11   9718
   6   186289
    11   117401
conditional tree for: set(['11', '3'])
  Null Set   1
   6   117401
conditional tree for: set(['11'])
  Null Set   1
   6   261773


接下来看下有多少新闻报道或报道集合曾经被10万或者更多的人浏览过：

In [28]:
print len(frequentList)

9


总共有9个。下面看看都是那些：

In [29]:
frequentList

[{'1'},
 {'1', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'11'},
 {'11', '6'},
 {'6'}]